# *GMOnotebook*
#### **Notebook template for applying routine hyperspectral/segmentation cross-analysis phenomics workflow over new datasets**

###### Notebook template v0.2.03 (Oct 14, 2020)
###### v0.2.0 onward applies the Python `GMOdetector` package instead of the deprecated R `GMOdetectoR` package, for much faster runtime
###### v0.2.01 has calls to `conda` cleaned up with `conda activate` and `conda deactivate` in the same code blocks to avoid any problems with warnings/errors preventing automatic execution of subsequent blocks
###### v0.2.02 is cleaned up using path variables (declared in notebook) instead of hardcoded paths, for portability
###### v0.2.03 is more clean to avoid running alignment-processed images through `inference.py` if an analysis if being repeated... also has better instructions

v.0.2.05 is updated to work with FAA wk7. Specifically, there are updates to deal with increased image resolution (y~=2000), off-center images, and rotated images, including metadata normalization and user inputs for parameterized grid cropping.

<img src="Figures/WorkflowFlowchart.png">

In this workflow, images taken with the macroPhor Array dual RGB/hyperspectral imaging platform are analyzed by a workflow in which regression quantifies fluorescent signals in hyperspectral images, deep learning segments RGB images into different tissues, and these datasets are cross-referenced to produce statistics on growth of transgenic callus and shoot.

#### Instructions for use
1.  Enter information for the experiment below
2. Set <font color=blue>variables</font> for data paths and parameters, as instructed by colored boxes.
3. "Save as" with filename describing experiment and anything special about this analysis (e.g. T18_OD_TAO_wk7_automation_test_attempt2.ipynb)
4. Run notebook from console, by either one of two methods:
- In the top bar of JupyterLab, select "Run" and "Run all cells"
- In a terminal, enter the below command with the notebook filename inserted<br>
```jupyter nbconvert --to HTML --ExecutePreprocessor.timeout=-1 --allow-errors --execute insert_filename_here```
5. Wait for email

#### Experiment ID and quick description:

<div class="alert alert-block alert-success">
Provide a short description of the experiment in the below box. This should include unique identifier codes for the experiment, along with a short description of genotypes and treatments studied. The timepoint should also be included. </div>

#### Parameters for analysis:

<div class="alert alert-block alert-success">
The below variables must be modified appropriately every time this workflow is run over new images.
</div>

##### Data location
The `data` variable below provides the **complete** path to the folder containing data to be analyzed. This should include all folders and subfolders in which the data of interest is organized by. For the organizational system used for our lab's data, this should follow the format "/Experiment/Subexperiment/Timepoint/"

In [1]:
data_prefix="/run/media/labgroup/Elements_9/"

In [2]:
data="${data_prefix}/Transformation/FAE_Dev_Genes/wk7/Fluorescence/"

##### Sample information
Every experiment has a randomization datasheet, which was used to organize treatment and genotype information for each plate, prepare labels, and randomize plates. The path to this file is provided through the `randomization_datasheets` variable below. This workflow requires this datasheet in order to know which plates have which genotype/treatment. At a later date, we will integrate an ability to read this data directly from labels.

In [116]:
randomization_datasheet="/run/media/labgroup/Elements_9/Transformation/FAE_Dev_Genes/FAE_randomization_list_labels.xlsx"

##### Exclusion of missing/contaminated explants from analysis

Set the `missing_explants` variable to `"Automatic"` if using model to automatically detect missing and contaminated explants. Note that this model is only supported for plates with 12 explants. Otherwise, provide an appropriately formatted `.csv` spreadsheet (see example)  of manually scored contamination / missing explant data.

In [4]:
missing_explants="Automatic"

Enter your email where results will be sent

In [5]:
email=michael.nagle@oregonstate.edu
















<div class="alert alert-block alert-warning">
All variables below should be modified only as needed to indicate the fluorescent proteins in samples and the grid layout of explants. </div>

##### Provide information pertinent to the fluorophores in the sample

Below should be all known fluorescent components contained in the sample. This includes each fluorescent protein, as well as a "noise" or "diffraction" term if applicable. All of these components must exist in the user's spectral library. `GMOdetector` currently comes with a spectral library that includes, by default:
- DsRed
- ZsYellow
- GFP
- Chl
- ChlA
- ChlB
- Diffraction

In [ ]:
fluorophores=(DsRed Chl Diffraction) # An explanation of array variables in bash is here: https://tldp.org/LDP/Bash-Beginners-Guide/html/sect_10_02.html

The user has the option of limiting loading of hyperspectral data and subsequent regression to a specific range of wavelengths, using the `desired_wavelength_range` array variable. This range should cover all fluorophores provided in the `fluorophores` array variable able. Aside from runtime, there is no disadvantage to including a wider range than is needed.

In [ ]:
desired_wavelength_range=(500 900)

##### To assist user inspection of regression results, false color plots will be produced by `GMOdetector` to show results of regression over whole samples. 
###### Note: In v0.2.0 of the workflow, these parameters are independent of those used later by `GMOlabeler` to produce by-explant plots including false color plots. These will be made the same in a later update.
The `FalseColor_channels` array variable indicates the components to be plotted as red, green and blue, in that order.

In [ ]:
FalseColor_channels=(Chl DsRed Diffraction)

The `FalseColor_caps` array variable indicates an upper limit of signal for each of these component. Any signal at or above these values will appear with maximum brightness; thus, these variables are comparable to exposure on a camera. If caps are too high, not much signal at lower ranges will be seen. If cap for a given component is too low, the false color images will appear overexposed with respect to the component.

In [ ]:
FalseColor_caps=(200 400 50)

##### `GMOlabeler` will use the below parameters to classify individual explants on plates as transgenic or not.

In [ ]:
#reporter=GFP
reporter=DsRed

Parameters for reporter signal threshold and pixel threshold must be provided by the user. Our grid search yielded several noted below. These were most recently calculated from statistics produced with Python `GMOdetector`.

<img src="Figures/GMOlabeler_parameters.png">

In [ ]:
reporter_threshold=38
pixel_threshold=3

##### Grid information

Currently, `GMOlabeler` distinguishes explants by their expected position on a plate. This is supported for plates with 12 or 20 explants, arranged in specific positions on a grid. False positives may result in cases in which one explant grows to such an extent that it intrudes into the grid square meant for an adjacent explant. <br>In the future, an additional neural network will be used to segment individual explants on a per-pixel basis, avoiding the need for crude cropping, and the outputs from the resulting model will be integrated with `GMOlabeler`.

Cropping to grid squares is supported for standard grids with 12 or 20 explants, as indicated by the `grid` variable. An image of each grid square (with a file path provided by the `grid_file` variable) will be superimposed over each image during `GMOlabeler` analysis to allow the user to easily inspect and verify grid positions.

In [ ]:
grid=12

<div class="alert alert-block alert-danger">
The below variables do not need to be modified during any routine use of the workflow.
</div>

For the following variables, use 0 (False) or 1 (True)

In [ ]:
composite=0

Set dimensions for plot outputs

In [ ]:
#width=15
width=9
height=5

In [ ]:
parallel=0

#### Paths to workflow components on local machine

In [ ]:
cwd="/home/labgroup/code/GMOnotebook"

In [ ]:
#gmodetector_wd="/scratch2/NSF_GWAS/notebooks/gmodetector_py/"
gmodetector_wd="/home/labgroup/code/gmodetector_py/"
spectral_library_path="${gmodetector_wd}spectral_library/"
#output_directory_prefix="/scratch2/NSF_GWAS/gmodetector_out/"
#deeplab_path="/scratch2/NSF_GWAS/deeplab/"
deeplab_path="/home/labgroup/code/deeplab/"
#alignment_path="/scratch2/NSF_GWAS/ImageAlignment/"
alignment_path="/home/labgroup/code/ImageAlignment/"
#gmolabeler_path="/scratch2/NSF_GWAS/GMOlabeler/"
gmolabeler_path="/home/labgroup/code/GMOlabeler/"
#contamination_path="/scratch2/NSF_GWAS/Contamination"
contamination_path="/home/labgroup/code/DenseNet"
data_prefix="/home/labgroup/data/"
output_directory_prefix="${data_prefix}gmodetector_out/"

In [ ]:
#grid_file="/scratch2/NSF_GWAS/macroPhor_Array/grids/grid20_post_processed.png"
#grid_file="/scratch2/NSF_GWAS/macroPhor_Array/grids/grids_left_facing_125208_1_0_1_rgb_processed.jpg"

# For data from before when imager went off-center
# grid_file="${gmolabeler_path}/grids/grid_12_explants_left-facing_processed.jpg"

# For offcenter images
grid_file="${alignment_path}/output/chromagrid_I5.0_F1.9_L100_122503_0_0_0_rgb_aligned.jpg"

<div class="alert alert-block alert-info">
With all above variables set, please "Save as..." with a filename referencing this specific dataset. <br>Finally, deploy the workflow (Step 4 in above instructions).
</div>

# Automated workflow to be deployed

See the below code for a walkthrough of how GMOnotebook works, or view the outputs after running the workflow for help troubleshooting errors in specific steps of analysis.

<div class="alert alert-block alert-danger"> <b>Danger:</b> Do not modify any below code without creating a new version of the template notebook. During routine usage, this workflow should be customized only by modifying variables above, while leaving the below code unmodified. </div>

These internal variables are set automatically.

In [ ]:
datestamp=$(date +”%Y-%m-%d”)

In [ ]:
data_folder=$(echo $data | cut -d/ -f5-)

In [ ]:
timepoint="$(basename -- $data_folder)"

In [ ]:
output_directory_full="$output_directory_prefix$data_folder"

In [ ]:
dataset_name=$(echo $data_folder | sed -e 's/\//-/g')

##### Time analysis begins:

In [24]:
echo $(date)

Wed Jan 20 21:54:03 PST 2021


## 1. Quantification of fluorescent proteins by regression

The Python package `GMOdetector` is used to quantify fluorescent proteins in each pixel of hyperspectral images via linear regression. Hyperspectral images are regressed over spectra of known components, and pixelwise maps of test-statistics are constructed for each component in the sample. This approach to quantifying components of hyperspectral images is described in-depth in the Methods section from <a href="https://link.springer.com/article/10.1007/s40789-019-0252-7" target="_blank">Böhme, et al. 2019</a>. Code and documentation for `GMOdetector` is on <a href="https://github.com/naglemi/GMOdetector_py" target="_blank">Github</a>.

In [25]:
cd $gmodetector_wd

In [26]:
job_list_name="$dataset_name.jobs"

In [27]:
rm -rf $job_list_name

In [29]:
for file in $data/*.hdr
do
 if [[ "$file" != *'hroma'* ]]; then
  echo "python wrappers/analyze_sample.py \
--file_path $file \
--fluorophores ${fluorophores[*]} \
--min_desired_wavelength ${desired_wavelength_range[0]} \
--max_desired_wavelength ${desired_wavelength_range[1]} \
--red_channel ${FalseColor_channels[0]} \
--green_channel ${FalseColor_channels[1]} \
--blue_channel ${FalseColor_channels[2]} \
--red_cap ${FalseColor_caps[0]} \
--green_cap ${FalseColor_caps[1]} \
--blue_cap ${FalseColor_caps[2]} \
--spectral_library_path "$spectral_library_path" \
--output_dir $output_directory_full" >> $job_list_name
 fi
done

In [30]:
echo $job_list_name

Elements_9--Transformation-FAE_Dev_Genes-wk7-Fluorescence-.jobs


In [31]:
conda activate test-environment

if [ $parallel -eq 1 ]
then
    parallel -a $job_list_name >> $data/log_gmodetector.txt
fi

if [ $parallel -eq 0 ]
then
    bash $job_list_name >> $data/log_gmodetector.txt
fi

conda deactivate
conda deactivate

(test-environment) (test-environment) (test-environment) (test-environment) During startup - Warning messages:
1: package ‘methods’ was built under R version 3.6.1 
2: package ‘datasets’ was built under R version 3.6.1 
3: package ‘utils’ was built under R version 3.6.1 
4: package ‘grDevices’ was built under R version 3.6.1 
5: package ‘graphics’ was built under R version 3.6.1 
6: package ‘stats’ was built under R version 3.6.1 
wrappers/analyze_sample.py:188: UserWarning: Sample is not being normalized with a chroma standard.
  warnings.warn('Sample is not being normalized with a chroma standard.')
/home/labgroup/anaconda3/envs/test-environment/lib/python3.8/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)
/home/labgroup/anaconda3/envs/test-environment/lib/python3.8/sit

##### Time regression completes:

In [32]:
echo $(date)

Thu Jan 21 01:38:46 PST 2021


## 2. Neural networks to segment tissues, classify missing/contaminated explants

### 2.1. Semantic segmentation of tissues

Images are segmented into specific plant tissues by a deep neural network of the state-of-the-art Deeplab v3 architecture <a href="https://arxiv.org/abs/1706.05587" target="_blank">Liang-Chieh et al., 2017</a>. The model has been trained using training sets generated with our annotation GUI Intelligent DEep Annotator for Segmentation (IDEAS, available on <a href="https://bitbucket.org/JialinYuan/image-annotator/src/master/" target="_blank">Bitbucket</a>, publication pending). Our branch of the Deeplab v3 repo, including a Jupyter walkthrough for training, can be found on Github.

Training is completed upstream of this notebook, which only entails analysis of test data using the latest model.

<img src="Figures/downsized/segmentation_composite2.png">

Figure: This example image was taken from an experiment on the effects of different CIMs on cottonwood regeneration. This composite image illustrates that for every sample, tissues are segmented into stem (red), callus (blue) and shoot (green). These composite images, useful for manual inspection of results, are produced when the 'composite' option is on.

#### 2.1.1. Pre-processing
##### 2.1.1.1. Normalize orientation, crop to remove labels, and resize images to 900x600

We desire for images to all be in the same orientation. At one point, the camera on the *macroPhor Array* was set to automatically detect orientation, which led to images randomly being in portrait or landscape. Here we will standardize the orientation.

In [33]:
conda activate alignment
for filename in $data/*.jpg; do
    #echo $filename
    exiftool -Orientation=8 -n $filename
    done
conda deactivate

(alignment)     1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated
    1 image files updated


In [34]:
rm -f $data/*original*

This script resizes images to 900x900 and then crops away top and bottom 150 pixels for a final image size of 900x600.

In [35]:
cd ${cwd}/intermediates/

Note, the `standardize_orientation.py` script was used for a situation where image rotation was attempted manually in Mac OS X but metadata was left behind, causing downstream confusion. This approach is no longer needed since we are using `exiftool` (above) to make `Orientation` in metadata consistent.

In [36]:
conda activate base
#python standardize_orientation.py $data
python crop.py $data
conda deactivate

(base) (base) (base) 

##### 2.1.1.2. prepare test.csv file for inference

Make a list of all our image files

In [37]:
cd $data
ls -d $PWD/* $data | grep -i "rgb_cropped.jpg" > test.csv

Remove the chroma standard from list of RGB image data to be segmented

In [38]:
sed -i '/hroma/d' "${data}/test.csv"

#### 2.1.2. Inference

The trained model is deployed to perform semantic segmentation of experimental images. A list of RGB images to be segmented by the trained model is passed through the --image-list option. For each of these images, we will obtain an output mask (.png) of labeled tissues

Dependencies include `opencv`, `scipy`, `yaml` and `tensorflow` (version 1.14)

In [39]:
cd $deeplab_path
conda activate deeplab
python -W ignore inference.py --image-list "${data}/test.csv" >> $data/log_inference.txt
conda deactivate
conda deactivate

(deeplab) WARNING:tensorflow:From inference.py:34: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



2021-01-21 02:07:55.769087: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX
2021-01-21 02:07:56.033456: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2494325000 Hz
2021-01-21 02:07:56.034855: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x562851455550 executing computations on platform Host. Devices:
2021-01-21 02:07:56.035025: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2021-01-21 02:08:01.653759: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to e

#### 2.3. Post-processing

Name outputs to reflect that they are segmentation results

In [40]:
echo $data

/run/media/labgroup/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence/


In [41]:
cd $data
for file in *_rgb_cropped.png; do mv -f "$file" "${file%_rgb_cropped.png}_segment_cropped.png"; done

Re-expand segment outputs to same size as original RGB files

Dependencies include `scikit-image`

In [42]:
conda activate alignment
cd $alignment_path
python expand.py $data >> $data/log_expand.txt
conda deactivate

(alignment) (alignment) (alignment) 

Make composite images with side-by-side RGB, segmentation outputs and blended images

In [43]:
echo $data

/run/media/labgroup/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence/


In [45]:
conda activate test-environment
if [ $composite -eq 1 ]
then
    cd $gmolabeler_path
    python image_blender.py $data 0.75 'both' 1 180 >> $data/log_blend.txt
fi
conda deactivate

(test-environment) (test-environment) 

#### 2.4. Classification of contaminated/missing explants

Plates are cropped into sub-images for each explant and each is analyzed to determine if the explant position should be excluded from analysis due to being missing or contamination. Missing and contaminated explants are recognized using a trained Densenet model (<a href="https://github.com/Contamination-Classification/DenseNet" target="_blank">Huang, et al. 2018</a>). Our fork of the Densenet repository is available on <a href="https://arxiv.org/abs/1608.06993" target="_blank">GitHub</a>.

<img src="Figures/Densenet.png">
Figure: These are four examples of contaminated explants used in the training set for this pre-trained model

If the mode for missing explant data is automatic, prepare input file for script to detect missing explants and run this script.

In [44]:
img_list_path="${data}/rgb_list.txt"

Dependencies include `keras-preprocessing`, `termcolor`,  `protobuf` and `absl-py`

In [45]:
echo $img_list_path

/run/media/labgroup/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence//rgb_list.txt


When running `inference.py` to detect missing or contaminated explants, the user should provide dimensions for cropping down to the grid borders. Note, these dimensions apply after the image is rescaled to 2000x2000. Default dimensions, used before the imager began to take "off-center" images, are (310, 515, 1750, 1610). Dimensions for cropping for the off-center images are (260, 600, 1700, 1710).

In [58]:
if [ $missing_explants = "Automatic" ]; then
    conda activate densenet
    echo "Missing explants will be inferred."
    cd $data
    ls -d $PWD/* $data | grep -i "rgb.jpg" > rgb_list.txt
    sed -i '/hroma/d' rgb_list.txt
    #sed -i 's/.jpg//g' rgb_list.txt
    #cat rgb_list.txt
    cd $contamination_path
    img_list_path="${data}/rgb_list.txt"
    #echo $img_list_path
    python inference.py \
    --img-list=$img_list_path \
    --crop_dims "(260,600,1700,1710)" \
    --output_file=output.csv >> $data/log_contam.txt
    mv -f output.csv "${data}/output.csv"
    missing_explants="${data}/output.csv"
    conda deactivate
else
    echo "Missing explants input manually by user, in file: "
    echo $missing_explants
fi

Missing explants will be inferred.
Using TensorFlow backend.
/home/labgroup/anaconda3/envs/densenet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/labgroup/anaconda3/envs/densenet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/labgroup/anaconda3/envs/densenet/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype(

## 3. Alignment of RGB and hyperspectral layers

To match the frame and angle of RGB and hyperspectral image layers, we apply a scale-invariant feature transformation (<a href="https://github.com/NSF-Image-alignment/ImageAlignment" target="_blank">GitHub</a>). Using a pair of standard images, a homography matrix is calculated for the necessary transformation of RGB images to align with hyperspectral images. The transformation can then be applied to large batches of images rapidly, as long as the RGB and hyperspectral cameras remain in the same positions relative to one another (as they do in the macroPhor Array platform)

<img src="Figures/Alignment.png">
Figure: To enable precise calculation of a homography matrix for transformation of RGB images to match hyperspectral images, we used images of a piece of paper with grid marks. These images are provided by the user inputs to --hyper-img and --rgb-img in the below call to the alignment script. If using a phenotyping platform other than the macroPhor Array, or using updated camera settings, these variables will need to be replaced.

#### 3.1. Prepare inputs for alignment

We need to produce a csv with two columns with headers `hyper_img` and `rgb_images`. For each RGB image being transformed in batch alignment (mode 2), we can test the alignment by producing superimposed images of the transformed RGB images together with a hyperspectral layer. The hyperspectral layer can either be for a grid (fast) or can be for a layer extracted from the hyperspectral image of each channel (slow, but useful for making sure a certain homography matrix works reliably to transform a batch of images).

Let's first generate the list that will go into the column `rgb_images`

In [47]:
cd $data
ls | grep -i 'rgb\.jpg' > file_list_part1.csv
ls | grep -i 'segment_uncropped\.png' > file_list_part2.csv
cat file_list_part* > file_list.csv
# sed -i '/hroma/d' file_list.csv
cwd2=$(pwd)/
awk -v prefix="$cwd2" '{print prefix $0}' file_list.csv > temp
mv -f temp file_list.csv
echo 'rgb_images' | cat - file_list.csv > temp && mv -f temp file_list.csv

Now to modify the list going into the `rgb_images` column to produce the list of filenames for `hyper_img`:

In [48]:
cp file_list.csv file_list_hyper_channel.csv
sed -i 's/_rgb.jpg/_hyperchannel.csv/g' file_list_hyper_channel.csv
sed -i 's/rgb_images/hyper_img/g' file_list_hyper_channel.csv
sed -i 's/_segment_uncropped.png/_hyperchannel.csv/g' file_list_hyper_channel.csv


Bind these columns together...

In [49]:
paste --delimiters=',' file_list_hyper_channel.csv file_list.csv > rgb_and_hyper_channel_lists.csv

#### 3.2 Run alignment

In [24]:
grid_hdr=$(ls $data | grep -i "grid" | grep -i "hdr")
grid_jpg=$(ls $data | grep -i "grid" | grep -i "rgb.jpg")

echo $grid_hdr
echo $grid_jpg

In [25]:
ls $data

FAE1_I5.0_F1.9_L100_143813_0_0_0_Fluorescence.hdr
FAE1_I5.0_F1.9_L100_143813_0_0_0_Fluorescence.raw
FAE1_I5.0_F1.9_L100_143813_0_0_0_hyperchannel.csv
FAE1_I5.0_F1.9_L100_143813_0_0_0_rgb_cropped.jpg
FAE1_I5.0_F1.9_L100_143813_0_0_0_rgb.jpg
FAE1_I5.0_F1.9_L100_143813_0_0_0_segment_cropped.png
FAE1_I5.0_F1.9_L100_143813_0_0_0_segment_uncropped.png
FAE1_I5.0_F1.9_L100_143859_1_0_1_Fluorescence.hdr
FAE1_I5.0_F1.9_L100_143859_1_0_1_Fluorescence.raw
FAE1_I5.0_F1.9_L100_143859_1_0_1_hyperchannel.csv
FAE1_I5.0_F1.9_L100_143859_1_0_1_rgb_cropped.jpg
FAE1_I5.0_F1.9_L100_143859_1_0_1_rgb.jpg
FAE1_I5.0_F1.9_L100_143859_1_0_1_segment_cropped.png
FAE1_I5.0_F1.9_L100_143859_1_0_1_segment_uncropped.png
FAE1_I5.0_F1.9_L100_143959_2_0_2_Fluorescence.hdr
FAE1_I5.0_F1.9_L100_143959_2_0_2_Fluorescence.raw
FAE1_I5.0_F1.9_L100_143959_2_0_2_hyperchannel.csv
FAE1_I5.0_F1.9_L100_143959_2_0_2_rgb_cropped.jpg
FAE1_I5.0_F1.9_L100_143959_2_0_2_rgb.jpg
FAE1_I5.0_F1.9_L100_143959_2_0_2_segment_cropped.png
FAE1_I5.0_F

##### Generate `csv` of a hyperspectral channel for the grid standard

In [51]:
conda activate test-environment
cd $alignment_path
python hypercube_to_csv.py \
$data$grid_hdr \
700 \
130 \
$data"hypercube_to_csv.csv" \
$data"hypercube_to_csv.jpg"
conda deactivate

(test-environment) (test-environment) Reading hyperspectral image /run/media/labgroup/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence/
Traceback (most recent call last):
  File "hypercube_to_csv.py", line 40, in <module>
    main(hdr_path = sys.argv[1],
  File "hypercube_to_csv.py", line 7, in main
    image1 = envi.open(hdr_path)
  File "/home/labgroup/anaconda3/envs/test-environment/lib/python3.8/site-packages/spectral/io/envi.py", line 289, in open
    header_path = find_file_path(file)
  File "/home/labgroup/anaconda3/envs/test-environment/lib/python3.8/site-packages/spectral/io/spyfile.py", line 120, in find_file_path
    raise FileNotFoundError(msg)
spectral.io.spyfile.FileNotFoundError: Unable to locate file "/run/media/labgroup/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence/". If the file exists, use its full path or place its directory in the SPECTRAL_DATA environment variable.
(test-environment) 

##### Run alignment to generate homography matrix

In [52]:
aligned=$(echo $grid_jpg | sed 's/rgb/rgb_aligned/g')

In [53]:
echo $data"hypercube_to_csv.csv"
echo $data$grid_jpg

/run/media/labgroup/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence/hypercube_to_csv.csv
/run/media/labgroup/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence/


In [54]:
conda activate alignment
cd $alignment_path
python main.py \
--hyper-img $data"hypercube_to_csv.csv" \
--rgb-img $data$grid_jpg \
--mode 1 \
--image_thresh_high=120 \
--image_thresh_low=50 \
--distance=0.6 \
--gaussian_sigma 0.55
conda deactivate

(alignment) (alignment) Traceback (most recent call last):
  File "main.py", line 215, in <module>
    main(args)
  File "main.py", line 57, in main
    hyp_img = genfromtxt(hyper_img_path, delimiter=',')
  File "/home/labgroup/anaconda3/envs/alignment/lib/python3.7/site-packages/numpy/lib/npyio.py", line 1749, in genfromtxt
    fid = np.lib._datasource.open(fname, 'rt', encoding=encoding)
  File "/home/labgroup/anaconda3/envs/alignment/lib/python3.7/site-packages/numpy/lib/_datasource.py", line 195, in open
    return ds.open(path, mode, encoding=encoding, newline=newline)
  File "/home/labgroup/anaconda3/envs/alignment/lib/python3.7/site-packages/numpy/lib/_datasource.py", line 535, in open
    raise IOError("%s not found." % path)
OSError: /run/media/labgroup/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence/hypercube_to_csv.csv not found.
(alignment) 

In [55]:
display < output/$aligned

cat: -: Is a directory
Not a valid image: b''

In [56]:
# conda activate alignment
# python main.py \
# --hyper-img '/home/labgroup/Downloads/input/chromagrid_I5.0_F1.9_L100_131132_0_0_0_rgb.csv' \
# --rgb-img '/home/labgroup/Downloads/input/chromagrid_I5.0_F1.9_L100_131132_0_0_0_rgb.jpg' \
# --mode 1 \
# --image_thresh_high=120 \
# --image_thresh_low=50 \
# --distance=0.6 \
# --gaussian_sigma 0.57
# conda deactivate

##### Generate `csv` of hyperspectral channel for every image in dataset

In [57]:
hdr_files=$data/*.hdr

In [58]:
cd $alignment_path
conda activate test-environment
for hdr_file in $hdr_files
do
 csv_file=$(sed 's/Fluorescence.hdr/hyperchannel.csv/g' <<< $hdr_file)
 csv_file=$(sed 's/.hdr/hyperchannel.csv/g' <<< $csv_file)
 csv_jpg=$(sed 's/csv/jpg/g' <<< csv_file)
 python hypercube_to_csv.py \
 $hdr_file \
 500 \
 200 \
 $csv_file \
 $csv_jpg
done
conda deactivate

(test-environment) Reading hyperspectral image /run/media/labgroup/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence//FAE1_I5.0_F1.9_L100_143813_0_0_0_Fluorescence.hdr
/home/labgroup/anaconda3/envs/test-environment/lib/python3.8/site-packages/spectral/io/envi.py:175: UserWarning: Parameters with non-lowercase names encountered and converted to lowercase. To retain source file parameter name capitalization, set spectral.settings.envi_support_nonlowercase_params to True.
  warnings.warn(msg)
Extracting channel at index 200
Writing channel matrix to /run/media/labgroup/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence//FAE1_I5.0_F1.9_L100_143813_0_0_0_hyperchannel.csv
Making image for inspection and writing to jpg_file
Reading hyperspectral image /run/media/labgroup/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence//FAE1_I5.0_F1.9_L100_143859_1_0_1_Fluorescence.hdr
/home/labgroup/anaconda3/envs/test-environment/lib/python3.8/site-packages/spectral/io/envi.py:175: U

##### Run alignment to apply homography matrix to all images

In [27]:
conda activate alignment
cd $alignment_path
file_list_input="${data}/rgb_and_hyper_channel_lists.csv"
python main.py \
--hyper-img "/run/media/labgroup/Elements_9//Transformation/FAD_Dev_Genes/wk7/Fluorescent/hypercube_to_csv.csv" \
--img-csv $file_list_input \
--mode 2 \
--h_matrix_path "output/chromagrid_I5.0_F1.9_L100LED30_125827_0_0_0_rgb_homography.npy" >> $data/log_alignment_mode2.txt
conda deactivate
conda deactivate

(alignment) (alignment) (alignment) (alignment) (base) 

If using this method, we should have RGB and hyperspectral images of a grid. Here, we will assume they have "grid" in their name and can be found in the same folder as the sample images.

We will now extract a channel from the grid hypercube.

Note, for recent "chromagrid" images, alignment requires the parameter `--gaussian_sigma 2.5` be added.

##### 3.1.2. Generate homography matrix for alignment

Below block generates homography matrix from a pair of an archived RGB / hyperspectral images channel layers we know to align well with these parameters.

##### Apply homography matrix to transform a batch of images

In [28]:
# conda activate alignment
# cd $alignment_path
# file_list_input="${data}/file_list.csv"
# python main.py \
# --img-csv $file_list_input \
# --mode 2 \
# --h_matrix_path "output/chromagrid_I5.0_F1.9_L100_131132_0_0_0_rgb" >> $data/log_alignment.txt
# conda deactivate
# conda deactivate

Below commented-out part is for if we are re-doing alignment every time instead of taking a standard pair of images that align well.

In [29]:
aligned=$(echo $grid_jpg | sed 's/rgb/rgb_aligned/g')

In [30]:
# conda activate alignment
# python main.py \
# --hyper-img $data"hypercube_to_csv.csv" \
# --rgb-img $data$grid_jpg \
# --mode 1 \
# --image_thresh_high=120 \
# --image_thresh_low=50 \
# --distance=0.6 \
# --gaussian_sigma 0.649
# conda deactivate

In [31]:
# display < $alignment_path/output/$aligned

In [32]:
# conda activate alignment
# cd $alignment_path
# file_list_input="${data}/file_list.csv"
# python main.py \
# --hyper-img $data"hypercube_to_csv.csv" \
# --rgb-img $data$grid_jpg \
# --img-csv $file_list_input \
# --mode 2 \
# --image_thresh_high=120 \
# --image_thresh_low=50 \
# --distance=0.6 \
# --gaussian_sigma 0.57 >> $data/log_align.txt
# conda deactivate
# conda deactivate

## 4. Cross-analyze deep segmentation and regression results

Scripts in the <a href="https://github.com/naglemi/GMOlabeler" target="_blank">GMOlabeler repository</a> are used to cross-reference results from deep segmentation of RGB images and regression of hyperspectral imaging, apply thresholding parameters to classify tissues as transgenic or escapes, and produce plots.

<img src="Figures/GMOlabeler.png">

Figure: The various steps of data processing in GMOlabeler are illustrated for an example explant from an experiment on CIM optimization for cottonwood. Images of plates are cropped to a sub-image for each explant. RGB segmentation results and hyperspectral regression results are cross-referenced to calculate fluorecent proteins in specific tissues and infer whether these tissues are transgenic.

##### 4.1. Prepare sample datasheet input

Prepare input file we will use for making plots. This file contains paths to CLS results, RGB images, and hyperspectral images.

In [33]:
echo $data

/run/media/labgroup/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence/


In [34]:
echo $data$aligned

/run/media/labgroup/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence/


In [35]:
conda activate gmolabeler
cd "${cwd}/intermediates/"
Rscript pre_label.R \
-r "${data}/" \
-R "${output_directory_prefix}" \
-i 1 \
-d $datestamp
conda deactivate

(gmolabeler) (gmolabeler) [1] "”2021-01-21”"
[1] "Looking for CLS data in: /home/labgroup/data/gmodetector_out/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence//"
[1] "2021-01-21"
[1] "Looking for CLS files in directory /home/labgroup/data/gmodetector_out/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence//"
[1] "How many CLS files? 46"
[1] "We will only include in analysis the FINAL of multiple images for FAE1_1_6"
[1] "Writing 45 rows to /run/media/labgroup/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence///samples_pre_labeling.csv"
(gmolabeler) 

##### 4.2. Cross-reference RGB and hyperspectral data

On 1/11/20, we altered `gmolabeler` to accept user-defined borders of the grid box, rather than hardcoding these borders in for specific grids with 12 or 20 explants. Thus, if providing `grid_borders`, `grid` can be left as `None`, or vice versa. To determine the borders, open the *aligned* grid file in a GUI like GIMP or Microsoft Paint and use the crosshairs to find the pixel positions of each border. These are then provided in order "left right bottom top".

# NOTE THE CHANGE HERE for FAE

In [46]:
data_old=/run/media/labgroup/Elements_9//Transformation/FAD_Dev_Genes/wk7/Fluorescent/

In [47]:
aligned_grid=$(ls $data_old | grep -i "grid" | grep -i "rgb_processed.jpg")

In [48]:
#aligned_grid=$(ls $data | grep -i "grid" | grep -i "rgb_processed.jpg")

In [49]:
echo $aligned_grid

chromagrid_I5.0_F1.9_L100LED30_125827_0_0_0_rgb_processed.jpg


Some notes on grid copping parameters:
- Submit a space-delimited string with parameters for borders of grid as "left right bottom top"
- For FAA wk3, use parameters "139 1580 1250 312"
- For FAA wk7 and FAD wk7, use "156 1600 1280 335"

In [50]:
conda activate gmolabeler
cd $gmolabeler_path
python main.py \
"${data}/samples_pre_labeling.csv" \
$data_old$aligned_grid \
$reporter_threshold \
$reporter \
$grid \
"hdf" \
$gmolabeler_path \
"156 1600 1280 335"
conda deactivate

(gmolabeler) (gmolabeler) Grid type 12
Loading plate 0 of 45
/run/media/labgroup/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence///FAE1_I5.0_F1.9_L100_152151_0_1_6_segment_uncropped_processed.png
Plate segment loaded
Plate RGB loaded
Loading CLS data from path/home/labgroup/data/gmodetector_out/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence///FAE1_I5.0_F1.9_L100_152151_0_1_6_Fluorescence_weights.hdf
Loading plate 1 of 45
/run/media/labgroup/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence///FAE2_I5.0_F1.9_L100_151757_18_2_6_segment_uncropped_processed.png
Plate segment loaded
Plate RGB loaded
Loading CLS data from path/home/labgroup/data/gmodetector_out/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence///FAE2_I5.0_F1.9_L100_151757_18_2_6_Fluorescence_weights.hdf
Loading plate 2 of 45
/run/media/labgroup/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence///FAE2_I5.0_F1.9_L100_151657_17_2_4_segment_uncropped_processed.png
Plate segment loaded


##### 4.3. Calculate sums of statistics over combined segments

We are interested in all regenerated tissue (callus + shoot) as well as all tissue (including stem as well). We will calculate aggregate statistics over these groups.

In [51]:
conda activate gmolabeler
Rscript calculate_sum_stats_over_combined_segments.R \
--output_dir "${gmolabeler_path}/output/" \
--datapath "${data_folder}/"
conda deactivate

(gmolabeler) [1] "Writing output with sums statistics calculated over combined tissue segments to: /home/labgroup/code/GMOlabeler//output/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence//stats_with_sums_over_tissues.csv"
(gmolabeler) 

##### 4.4. Make plots of results

In [117]:
conda activate gmolabeler
cd $gmolabeler_path
Rscript grid_item_plots.R \
-d "${data_folder}/" \
-r "$randomization_datasheet" \
-p $pixel_threshold \
-v categorical \
-m 1 \
-M $missing_explants \
-g $grid \
--sort 1 \
--height $height \
--width $width
conda deactivate

(gmolabeler) (gmolabeler) Warning message:
In storage.mode(default) <- type : NAs introduced by coercion
[1] "Saving list of input arguments to : /home/labgroup/code/GMOlabeler/plots/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence//gridplot_args.rds"
[1] "Reading in output from GMOlabeler at path: /home/labgroup/code/GMOlabeler/output/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence//stats_with_sums_over_tissues.csv"

[1] "Rows in output from GMOlabeler: 2491"

[1] "Max n_pixels_passing_threshold in output from GMOlabeler: 59483"

[1] "Max total_signal in output from GMOlabeler: 22149221.7910449"

[1] "Look at the top of output from GMOlabeler"
[1] ""                                                                                                                    
[2] "/run/media/labgroup/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence///FAE1_I5.0_F1.9_L100_152151_0_1_6.png"
[3] "/run/media/labgroup/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence

## 5. Email plots to user

##### 5.1. ZIP results

In [118]:
echo "${gmolabeler_path}/plots${data_folder}"

/home/labgroup/code/GMOlabeler//plotsElements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence/


In [119]:
cd "${gmolabeler_path}/plots/${data_folder}"

In [120]:
rm -f ./plants_over_plates.csv

In [121]:
echo ${gmolabeler_path}/output/${data_folder}/

/home/labgroup/code/GMOlabeler//output/Elements_9//Transformation/FAE_Dev_Genes/wk7/Fluorescence//


In [122]:
cp "${gmolabeler_path}/output/${data_folder}/plants_over_plates.csv" ./

In [123]:
cp "${gmolabeler_path}/output/${data_folder}/stats_with_sums_over_tissues.csv" ./

In [124]:
cd $data

In [125]:
rm -f Rplots.pdf

In [126]:
cd ../

This messy substitution is explained here: https://superuser.com/questions/1068031/replace-backslash-with-forward-slash-in-a-variable-in-bash

In [127]:
data_folder_Compress="${data_folder////-}.zip"
data_folder_Compress=${data_folder_Compress#?};

In [128]:
ls

Broadband  Fluorescence


In [129]:
cd $timepoint
zip -r $data_folder_Compress ./*

  adding: FAE1_I5.0_F1.9_L100_143813_0_0_0_Fluorescence.hdr (deflated 59%)
  adding: FAE1_I5.0_F1.9_L100_143813_0_0_0_Fluorescence.raw (deflated 58%)
  adding: FAE1_I5.0_F1.9_L100_143813_0_0_0_hyperchannel.csv (deflated 96%)
  adding: FAE1_I5.0_F1.9_L100_143813_0_0_0_rgb_cropped.jpg (deflated 0%)
  adding: FAE1_I5.0_F1.9_L100_143813_0_0_0_rgb.jpg (deflated 1%)
  adding: FAE1_I5.0_F1.9_L100_143813_0_0_0_rgb_processed.jpg (deflated 0%)
  adding: FAE1_I5.0_F1.9_L100_143813_0_0_0_segment_cropped.png (deflated 4%)
  adding: FAE1_I5.0_F1.9_L100_143813_0_0_0_segment_uncropped.png (deflated 39%)
  adding: FAE1_I5.0_F1.9_L100_143813_0_0_0_segment_uncropped_processed.png (deflated 11%)
  adding: FAE1_I5.0_F1.9_L100_143859_1_0_1_Fluorescence.hdr (deflated 59%)
  adding: FAE1_I5.0_F1.9_L100_143859_1_0_1_Fluorescence.raw (deflated 58%)
  adding: FAE1_I5.0_F1.9_L100_143859_1_0_1_hyperchannel.csv (deflated 96%)
  adding: FAE1_I5.0_F1.9_L100_143859_1_0_1_rgb_cropped.jpg (deflated 0%)
  adding: FAE1_I5

##### 5.2. Write email

In [130]:
duration=$(( SECONDS - start ))

https://unix.stackexchange.com/questions/53841/how-to-use-a-timer-in-bash

In [131]:
rm -f "${gmolabeler_path}/email_to_send.txt"
cp "${gmolabeler_path}/email_to_send_template.txt" "${gmolabeler_path}/email_to_send.txt"

In [132]:
echo "" >> "${gmolabeler_path}/email_to_send.txt"
echo "Number of samples run: " >> "${gmolabeler_path}/email_to_send.txt"

In [133]:
cat "${data}/test.csv" | wc -l >> "${gmolabeler_path}/email_to_send.txt"
echo "" >> "${gmolabeler_path}/email_to_send.txt"

In [134]:
if (( $SECONDS > 3600 )) ; then
    let "hours=SECONDS/3600"
    let "minutes=(SECONDS%3600)/60"
    let "seconds=(SECONDS%3600)%60"
    echo "Completed in $hours hour(s), $minutes minute(s) and $seconds second(s)" >> "${gmolabeler_path}/email_to_send.txt"
elif (( $SECONDS > 60 )) ; then
    let "minutes=(SECONDS%3600)/60"
    let "seconds=(SECONDS%3600)%60"
    echo "Completed in $minutes minute(s) and $seconds second(s)" >> "${gmolabeler_path}/email_to_send.txt"
else
    echo "Completed in $SECONDS seconds" >> "${gmolabeler_path}/email_to_send.txt"
fi

In [135]:
echo "" >> "${gmolabeler_path}/email_to_send.txt"

##### 5.3. Send email with results to user

In [136]:
pwd

/run/media/labgroup/Elements_9/Transformation/FAE_Dev_Genes/wk7/Fluorescence


In [137]:
conda deactivate

In [ ]:
mail -a $data_folder_Compress -s $data_folder $email < "${gmolabeler_path}/email_to_send.txt"

In [ ]:
du -sh $data_folder_Compress

In [ ]:
cat "${gmolabeler_path}/email_to_send.txt"

##### Time analysis ends

In [ ]:
echo $(date)